# Fines for providers in nursing homes
## Introduction

In this notebook, we are working with nursing home inspection data from the United States to try to predict which providers may be fined and for how much. We are going to use scikit-learn Python package to construct construct our machine learning models.


## Downloading the data

We can download the data set from Amazon S3:

In [1]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

--2021-12-27 01:12:49--  http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.95.193
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.95.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3398120 (3.2M) [text/csv]
Saving to: ‘./ml-data/providers-train.csv’

     0K .......... .......... .......... .......... ..........  1%  209K 16s
    50K .......... .......... .......... .......... ..........  3%  442K 11s
   100K .......... .......... .......... .......... ..........  4% 1.29M 8s
   150K .......... .......... .......... .......... ..........  6% 1.25M 7s
   200K .......... .......... .......... .......... ..........  7%  745K 6s
   250K .......... .......... .......... .......... ..........  9% 1.26M 5s
   300K .......... .......... .......... .......... .......... 10% 1.29M 5s
   350K .......... .......... 

In [2]:
import numpy as np
import pandas as pd

In [3]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [6]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [4]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

    def fit(self, X, y):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.group_averages['global_average'] = y.mean()
        return self

    def predict(self, X):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        # Return a list of predicted penalties based on group of samples in X
        return [self.group_averages.get(group, self.group_averages['global_average']) for group in X[self.grouper]]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [7]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(steps=[('sme', GroupMeanEstimator(grouper='STATE'))])

Next we should test that our predict method works.

In [8]:
state_model.predict(data.sample(5))

[8438.121359223302,
 9216.964686998395,
 8054.977611940299,
 8214.822977725675,
 16612.338211382114]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [9]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

In [12]:
from sklearn.impute import SimpleImputer

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer())
])

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {'estimator__n_estimators': range(20, 100, 10),
              'estimator__max_depth': range(3, 15, 1)}

simple_features_model = Pipeline([
    ('simple', simple_features),
    ('estimator', RandomForestClassifier(n_estimators=60, max_depth = 5))
])

gs = RandomizedSearchCV(simple_features_model, param_distributions = param_grid, cv = 5, n_iter = 50, n_jobs = 4, verbose = 1)

gs.fit(data, fine_counts > 0)

gs.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


{'estimator__n_estimators': 20, 'estimator__max_depth': 4}

In [15]:
simple_features_model = Pipeline([
    ('simple', simple_features),
    ('estimator', RandomForestClassifier(n_estimators=200, min_samples_leaf = 100))
])

simple_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('simple',
                 Pipeline(steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imputer', SimpleImputer())])),
                ('estimator',
                 RandomForestClassifier(min_samples_leaf=100,
                                        n_estimator

In [16]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

## Categorical features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator.

In [18]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [19]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    ('estimator', RandomForestClassifier(n_estimators=200, min_samples_leaf = 100))
])

In [20]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('categorical',
                 FeatureUnion(transformer_list=[('owner',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))])),
                                                ('cert',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['CERTIFICATION'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))]))])),
                ('estimator',
                 RandomForestClassifier(min_samples_leaf=100,
                                  

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`.

In [23]:
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

In [24]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [25]:
business_model = Pipeline([
    ('features', business_features),
    ('ploy', PolynomialFeatures(3)),
    ('estimator', RandomForestClassifier(n_estimators=200, min_samples_leaf = 100))
    
])

In [26]:
business_model.fit(data, fine_counts > 0)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                                                                                   'SPRINKLER_STATUS',
                                                       

## Survey results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [27]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X,pd.DataFrame):
               X = pd.DataFrame(X)
        return (pd.to_datetime( X[self.t2_col])
                -pd.to_datetime( X[self.t1_col])).values.reshape(-1,1).astype(float)

In [28]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [29]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    ("poly", PolynomialFeatures(2)),
    ("svd", TruncatedSVD(20) ),
    ("classifier", RandomForestRegressor(n_estimators=200, min_samples_leaf = 100))
])

In [31]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                                                                                                                   'CCRC_FACIL',
                                                                                                                                   'SFF',
       

*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*